# Grabbing Tithe Owners

#### Imports

In [1]:
import os
import re

import numpy as np
np.bool = np.bool_
import pandas as pd

from tqdm.auto import tqdm
tqdm.pandas()

import spacy
nlp = spacy.load('en_core_web_trf')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import platform
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'

c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\PhD\DissolutionProgramming\LND---Land-Paper\.venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


#### Loading

In [2]:
tdf = pd.read_csv(f'{RAW}/tithe_apportionments_1834_1852.csv')
tdf.columns = [x.lower() for x in tdf.columns]

# Little adjustments

tdf['area_perches'] = tdf['acres'] * 160 + tdf['roods'] * 40 + tdf['perches']
tdf = tdf[tdf['area_perches'] > 0]
tdf.rename(columns={'cultivat_1': 'cultivation'}, inplace=True)
tdf = tdf[['parish', 'landowner', 'occupier', 'cultivation', 'area_perches']]

C:\Users\nicho\AppData\Local\Temp\ipykernel_139012\3109805656.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv(f'{RAW}/tithe_apportionments_1834_1852.csv')


#### Grabbing Owners

In [3]:
trustees = re.compile(r'Trustees (under the will )?of (the )?(late )?(deceased )?', re.IGNORECASE)

executors = re.compile(r'Executors (under the will )?of (the )?(late )?(deceased )?', re.IGNORECASE)

representatives = re.compile(r'Representatives (under the will )?of (the )?(late )?(deceased )?', re.IGNORECASE)

nobility = re.compile(r'(Earl|Duke|Duchess|Countess|Viscount|Viscountess|Baron|Baroness) of [A-Z][a-z]+', re.IGNORECASE)

oh_shit = 0
for i, row in tqdm(tdf.iterrows(), total=len(tdf)):
    owner_surname = np.nan
    landowner_text = row['landowner']
    landowner_text = re.sub(trustees, '', landowner_text)
    landowner_text = re.sub(executors, '', landowner_text)
    landowner_text = re.sub(representatives, '', landowner_text)

    
    owner_doc = nlp(landowner_text)
    owner_ents = owner_doc.ents
    owner_ents_text = [x.text for x in owner_ents if x.label_ == 'PERSON']
    owner_ents_text = ' '.join(owner_ents_text)
    root = ''
    for token in owner_doc:
        if token.dep_ == 'ROOT':
            root = token.text
    if root not in owner_ents_text:
        noble = None
        noble = re.search(nobility, landowner_text)
        if noble:
            owner_surname = noble.group(0)
        else:
            oh_shit += 1
    else:
        owner_surname = root
    tdf.at[i, 'owner_surname'] = owner_surname

print(f'Share with no owner surname: {oh_shit/len(tdf)}')

  0%|          | 0/494732 [00:00<?, ?it/s]

Share with no owner surname: 0.04077156925365653


In [6]:
print(tdf.columns)

Index(['parish', 'landowner', 'occupier', 'cultivation', 'area_perches',
       'owner_surname'],
      dtype='object')


#### Saving

In [7]:
tdf.to_csv(f'{PROCESSED}/tithe_landowners.csv', index=False, encoding='utf-8')